Author: Jay Qi ([@jayqi](https://github.com/jayqi/))

This notebook demonstrates the functions in [`src/create_points.py`](../src/create_points.py). 

If using normally, it may be easier to instead use the `python -m src.create_points` command line interfance.

**Data setup:** Download [`Three_Rivers_Michigan_USA_line.zip`](https://drive.google.com/file/d/1fpI4I5KP2WyVD5PeytW_hoXZswOt0dwA/view?usp=drive_link) from the project Google Drive and save it to `data/raw/Three_Rivers_Michigan_USA_line.zip`.

In [1]:
from pathlib import Path

import geopandas as gpd

from src.create_points import remove_highways, create_points

In [2]:
proj_root = Path().resolve().parent
proj_root

PosixPath('/Users/jqi/repos/street-view-green-view')

## Load raw data

In [3]:
shpfile_dir = (
    proj_root
    / "data"
    / "raw"
    / "Three_Rivers_Michigan_USA_line.zip"
)
shpfile_dir

PosixPath('/Users/jqi/repos/street-view-green-view/data/raw/Three_Rivers_Michigan_USA_line.zip')

In [4]:
# Inspect raw data
gdf = gpd.read_file(shpfile_dir)
print(gdf.shape)
gdf.head()

(1203, 12)


,osm_id,osm_type,width,name,highway,layer,oneway,surface,tunnel,bridge,smoothness,geometry
0,628771085,ways_line,None,None,service,None,yes,asphalt,None,None,None,"LINESTRING (-85.65577 41.94215, -85.65435 41.9..."
1,628771094,ways_line,None,None,service,None,None,asphalt,None,None,None,"LINESTRING (-85.65436 41.94281, -85.65436 41.9..."
2,764612663,ways_line,None,None,service,None,None,None,None,None,None,"LINESTRING (-85.65178 41.94118, -85.65195 41.9..."
3,628771076,ways_line,None,None,service,None,yes,asphalt,None,None,None,"LINESTRING (-85.65612 41.94256, -85.65577 41.9..."
4,628771077,ways_line,None,None,service,None,None,asphalt,None,None,None,"LINESTRING (-85.65623 41.94256, -85.65612 41.9..."


In [5]:
# Visualize the raw data with folium
gdf.explore(color="purple")

## Remove highways

In [6]:
# Remove highways based on Treepedia criteria

print("Before:", gdf.shape)
print("Before:", sorted(gdf["highway"].unique()))
gdf = remove_highways(gdf)
print("After:", gdf.shape)
print("After:", sorted(gdf["highway"].unique()))

Before: (1203, 12)
Before: ['footway', 'path', 'primary', 'residential', 'secondary', 'service', 'tertiary', 'tertiary_link', 'track', 'trunk', 'trunk_link', 'unclassified']
After: (242, 12)
After: ['path', 'residential', 'track', 'unclassified']


In [7]:
# Visualize the data with highways removed with folium
gdf.explore(color="purple")

## Sample points

In [8]:
# Use create_points to interpolate points along the roads
print("Roads:", gdf.shape)
points_gdf = create_points(gdf)
print("Points:", points_gdf.shape)
points_gdf.head()

Roads: (242, 12)


Points: (4960, 3)


,osm_id,highway,geometry
0,17967640,residential,POINT (-85.65161 41.95364)
1,17967640,residential,POINT (-85.65155 41.95379)
2,17967640,residential,POINT (-85.65149 41.95394)
3,17967640,residential,POINT (-85.65142 41.95408)
4,17964573,residential,POINT (-85.65606 41.94811)


In [9]:
# Visualize all sampled points with folium
points_gdf.explore(
    color="red",
    marker_kwds=dict(radius=2),
)

## Inspect sampled points for a single road

In [10]:
# Get longest road row as an example
longest_road_osm_id = gdf.loc[gdf["geometry"].to_crs("EPSG:3857").length.idxmax()][
    "osm_id"
]
gdf.loc[gdf["osm_id"] == longest_road_osm_id]

,osm_id,osm_type,width,name,highway,layer,oneway,surface,tunnel,bridge,smoothness,geometry
256,203839666,ways_line,None,Hov Aire Drive,residential,None,None,None,None,None,None,"LINESTRING (-85.61024 41.95477, -85.61025 41.9..."


In [11]:
# Plot with folium. Overlay sampled points on top of the road line feature.
m = gdf[gdf["osm_id"] == longest_road_osm_id].explore(
    color="purple",
)
points_gdf[points_gdf["osm_id"] == longest_road_osm_id].explore(
    m=m,
    color="red",
    marker_kwds=dict(radius=2),
)